In [1]:
#Import packages
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import folium
#from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import anderson
from scipy.stats import norm
from matplotlib import pylab

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#import dataset created in Data Understanding
accidents = pd.read_csv(r"C:\Users\DETCAO03\V-Case study\02_Dataset\Used\Accidents.csv",low_memory=False, encoding='utf-8')

In [4]:
#Delete not necessary columns
accidents.drop(['Location_Easting_OSGR', 'Location_Northing_OSGR', 'LSOA_of_Accident_Location', 'Police_Force', 'Local_Authority_(District)',
                                   'Junction_Detail', '2nd_Road_Class', '2nd_Road_Number', 
                                   'Did_Police_Officer_Attend_Scene_of_Accident','Pedestrian_Crossing-Physical_Facilities'], axis=1, inplace=True)

In [6]:
#date
accidents['Date_time'] =  accidents['Date'] +' '+ accidents['Time']
accidents['Date_time'] = pd.to_datetime(accidents["Date_time"])
accidents.drop(['Date','Time'],axis =1 , inplace=True)

In [8]:
#Roadnumber
accidents["1st_Road_Class"] = accidents["1st_Road_Class"].map(lambda x: {1:"Motorway",2:"A(M)",3:"A",4:"B",5:"C", 6:"Unclassified"}.get(x,x))
accidents["1st_Road_Number"] = accidents["1st_Road_Number"].astype(str)
accidents["Roadnumber"] = accidents.apply(lambda x: "Unclassified" if x["1st_Road_Class"]=="Unclassified" else ("A"+x["1st_Road_Number"]+"(M)" if x["1st_Road_Class"]=="A(M)" else x["1st_Road_Class"][0] + x["1st_Road_Number"]), axis = 1)
accidents.drop(['1st_Road_Class','1st_Road_Number'],axis =1 , inplace=True)

In [9]:
accidents.head()

,Accident_Index,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Control,Pedestrian_Crossing-Human_Control,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Region,Date_time,Roadnumber
0,200501BS00001,-0.191170,51.489096,2,1,1,3,E09000020,A,3218,6,30,-1,0,1,2,2,0,0,1,London,2005-04-01 17:42:00,A3218
1,200501BS00002,-0.211708,51.520075,3,1,1,4,E09000020,B,450,3,30,2,0,4,1,1,0,0,1,London,2005-05-01 17:36:00,B450
2,200501BS00003,-0.206458,51.525301,3,2,1,5,E09000020,C,0,6,30,-1,0,4,1,1,0,0,1,London,2005-06-01 00:15:00,C0
3,200501BS00004,-0.173862,51.482442,3,1,1,6,E09000020,A,3220,6,30,-1,0,1,1,1,0,0,1,London,2005-07-01 10:35:00,A3220
4,200501BS00005,-0.156618,51.495752,3,1,1,2,E09000020,Unclassified,0,6,30,-1,0,7,1,2,0,0,1,London,2005-10-01 21:13:00,Unclassified


In [12]:
#fill missing region values with "mode" and binary encoding of region
#accidents["Region"] = accidents["Region"].fillna(accidents["Region"].mode()[0])

#drop missing values (151)
#accidents.dropna(inplace=True)

In [13]:
#mapping
accidents["Accident_Severity"] = accidents["Accident_Severity"].map(lambda x: {1: "Fatal",2:"Serious", 3:"Slight"}.get(x,x))
accidents["Road_Type"] = accidents["Road_Type"].map(lambda x: {1:"Roundabout", 2:"One way street", 3:"Dual carriageway",
                                                         6:"Single carriageway", 7:"Slip road", 9:"Unknown",
                                                         12:"One way street/Slip road", -1:"Missing data"}.get(x,x))
accidents["Pedestrian_Crossing-Human_Control"] = accidents["Pedestrian_Crossing-Human_Control"].map(lambda x: {0:"None within 50 metres",
                                                                                                       1:"Control by school crossing patrol",
                                                                                                       2:"Control by other authorised person",
                                                                                                       -1:"Data missing"}
                                                                                            .get(x,x))
accidents["Light_Conditions"] = accidents["Light_Conditions"].map(lambda x: {1:"Daylight", 4:"Darkness - lights lit", 5:"Darkness - lights unlit",
                                                                     6:"Darkness - no lighting", 7:"Darkness - lighting unknown",
                                                                     -1:"Data missing"}
                                                          .get(x,x))

accidents["Weather_Conditions"] = accidents["Weather_Conditions"].map(lambda x: {1:"Fine no high winds", 2:"Raining no high winds", 3:"Snowing no high winds",
                                                                         4:"Fine + high winds", 5:"Raining + high winds", 6:"Snowing + high winds",
                                                                         7:"Fog or mist", 8:"Other", 9:"Unknown", -1:"Data missing"}
                                                              .get(x,x))

accidents["Road_Surface_Conditions"] = accidents["Road_Surface_Conditions"].map(lambda x: {1:"Dry", 2:"Wet or damp", 3:"Snow", 4:"Frost or ice", 
                                                                                   5:"Flood over 3cm. deep", 6:"Oil or diesel", 7:"Mud",
                                                                                   -1:"Data missing"}
                                                                        .get(x,x))

accidents["Special_Conditions_at_Site"] = accidents["Special_Conditions_at_Site"].map(lambda x: {0:"None", 1:"Auto traffic signal - out",
                                                                                         2:"Auto signal part defective",
                                                                                         3:"Road sign or marking defective or obscured",
                                                                                         4:"Roadworks", 5:"Road surface defective", 6:"Oil or diesel",
                                                                                         7:"Mud", -1:"Data missing"}
                                                                              .get(x,x))

accidents["Carriageway_Hazards"] = accidents["Carriageway_Hazards"].map(lambda x: {0:"None",
                                                                           1:"Vehicle load on road", 2:"Other object on road", 3:"Previous accident",
                                                                           4:"Dog on road", 5:"Other animal on road", 
                                                                           6:"Pedestrian in carriageway - not injured",
                                                                           7:"Any animal in carriageway (except ridden horse)",
                                                                           -1:"Data missing"}
                                                                .get(x,x))
accidents["Urban_or_Rural_Area"] = accidents["Urban_or_Rural_Area"].map(lambda x: {1:"Urban", 2:"Rural", 3:"Unallocated"}.get(x,x))
accidents["Junction_Control"] = accidents["Junction_Control"].map(lambda x: {0:"Not at junction or within 20 metres",1:"Authorised person",2:"Auto traffic signal",3:"Stop sign",4:"Give way or uncontrolled",-1:"Data missing or out of range"}.get(x,x))
accidents["Day_of_Week"] = accidents["Day_of_Week"].map(lambda x: {1:"Sunday",2:"Monday",3:"Tuesday",4:"Wednesday",5:"Thursday",6:"Friday",7:"Saturday"}.get(x,x))

In [14]:
accidents.head()

,Accident_Index,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Local_Authority_(Highway),Road_Type,Speed_limit,Junction_Control,Pedestrian_Crossing-Human_Control,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Region,Date_time,Roadnumber
0,200501BS00001,-0.191170,51.489096,Serious,1,1,Tuesday,E09000020,Single carriageway,30,Data missing or out of range,None within 50 metres,Daylight,Raining no high winds,Wet or damp,None,None,Urban,London,2005-04-01 17:42:00,A3218
1,200501BS00002,-0.211708,51.520075,Slight,1,1,Wednesday,E09000020,Dual carriageway,30,Auto traffic signal,None within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,London,2005-05-01 17:36:00,B450
2,200501BS00003,-0.206458,51.525301,Slight,2,1,Thursday,E09000020,Single carriageway,30,Data missing or out of range,None within 50 metres,Darkness - lights lit,Fine no high winds,Dry,None,None,Urban,London,2005-06-01 00:15:00,C0
3,200501BS00004,-0.173862,51.482442,Slight,1,1,Friday,E09000020,Single carriageway,30,Data missing or out of range,None within 50 metres,Daylight,Fine no high winds,Dry,None,None,Urban,London,2005-07-01 10:35:00,A3220
4,200501BS00005,-0.156618,51.495752,Slight,1,1,Monday,E09000020,Single carriageway,30,Data missing or out of range,None within 50 metres,Darkness - lighting unknown,Fine no high winds,Wet or damp,None,None,Urban,London,2005-10-01 21:13:00,Unclassified


In [15]:
accidents.shape

(1780653, 21)

In [18]:
accidents.to_csv(r"C:\Users\DETCAO03\V-Case study\02_Dataset\Used\Dashboard_Data_Accidents.csv", index=False, encoding='utf-8')